# Mel Frequency Cepstrum Coefficients

In der heutigen Übung betrachten wir Mel Frequency Cepstrum Coefficients, die Standardmerkmale der Sprachverarbeitung. 

Wir führen zuerst ein k-Means-Clustern von Vokalen durch,
anschließend implementieren wir eine Geschlechtserkennung mit KNN.

Sie benoetigen zusaetzlich das Paket *soundfile* um Audiodateien zu lesen und 
*python_speech_features* zur MFCC-Berechnung.

# MFCC Berechnung

In [33]:
import matplotlib.pyplot as plt
import numpy as np
import os
import soundfile as sf
import python_speech_features as mfcc

Folgende Methode berechnet MFCCs auf einer Audiodatei mit Hilfe des Paketes python_speech_features und speichert das Ergebnis 

In [34]:
def extract_feats(audiof):
    """extract 20 dim mfcc features from an audio"""
    
    audio,samplerate= sf.read(audiof);
    mfcc_feat = mfcc.mfcc(audio, samplerate, 0.025, 0.01 , 20, appendEnergy=False)

    return mfcc_feat

## Geschlechtsklassifikation anhand von Sprachdaten


- Im Repository finden Sie Sprachdaten aus dem LibriSpeech-Corpus.
- Es existieren zwei Verzeichnisse mit Sprachdaten. Die Verzeichnisse sind in Unterverzeichnisse aufgeteilt. Jedes Unterverzeichnis beinhaltet Sprachdaten eines Sprechers, diese sind wiederum in Sessions unterteilt. In einer Session existieren mehrere Sprachdaten.
    - eines zum Training: *train-clean*
    - eines zum Testen: *test-clean-10spk*
- Es existeren utterance to speaker maps. Die Maps enthalten zwei Spalten. In der ersten Spalte steht die SprecherID, in der zweiten das label ('F' --> female, 'M' --> male) 
    - utt2spk.train
    - utt2spk.test-10spk





### Teil 1

Aufgaben:

1. Schreiben Sie Code, der die Sprachdaten einliest und MFCCs **pro Session eines Sprechers** berechnet. Am Ende erhalten Sie eine MFCC-Matrix pro Session.
2. Berechnen Sie den Mittelwert, die Varianz, sowie Minimum und Maximum der Merkmale einer Session.
3. Trainieren Sie mit diesen einen KNN-Klassifikator zur Geschlechtserkennung auf den *train-clean*-Daten
4. Testen Sie ihr System auf den *test-clean-10spk*-Daten

In [35]:
os.chdir("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech")
def gendermap(utt2spk):
    gender={}
    f=open(utt2spk, 'r')
    for line in f:
        line= ",".join( line.split(" ",1))
        columns=line.split(",")
        gender[columns[0]]=columns[1].strip()
    f.close()
    return gender
trainGenderMap=gendermap('utt2spk.train')
testGenderMap=gendermap('utt2spk.test-10spk')
print(testGenderMap)

{'1926': 'F', '6019': 'M', '458': 'M', '3168': 'M', '8312': 'F', '2893': 'M', '7517': 'F', '4088': 'F', '7780': 'F', '5867': 'F'}


In [36]:
labels_train=np.empty([0,1])
x_train=np.empty([0,4])

speakers = os.listdir("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech\\train-clean")
for s in speakers:
    sessions=os.listdir("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech\\train-clean\\"+s)
    for session in sessions:
        mfccMatrix = np.empty([0,20])
        files = os.listdir("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech\\train-clean\\"+s+"\\"+session)
        i=1
        for file in files:
            if(os.fsdecode(file).endswith('.flac')):
                mfccMatrix=np.vstack([mfccMatrix, extract_feats("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech\\train-clean\\"+s+"\\"+session+"\\"+file)])
        
        characteristics=np.array([np.mean(mfccMatrix), np.var(mfccMatrix), np.amin(mfccMatrix), np.amax(mfccMatrix)])
        x_train=np.vstack([x_train, characteristics])
        labels_train=np.append(labels_train,trainGenderMap[s])
print("Done")

Done


In [38]:
x_test=np.empty([0,4])
labels_test=np.empty([0,1])

speakers = os.listdir("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech\\test-clean-10spk")
for s in speakers:
    sessions=os.listdir("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech\\test-clean-10spk\\"+s)
    for session in sessions:
        mfccMatrix = np.empty([0,20])
        files = os.listdir("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech\\test-clean-10spk\\"+s+"\\"+session)
        i=1
        for file in files:
            if(os.fsdecode(file).endswith('.flac')):
                mfccMatrix=np.vstack([mfccMatrix, extract_feats("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech\\test-clean-10spk\\"+s+"\\"+session+"\\"+file)])
        
        characteristics=np.array([np.mean(mfccMatrix), np.var(mfccMatrix), np.amin(mfccMatrix), np.amax(mfccMatrix)])
        x_test=np.vstack([x_test, characteristics])
        labels_test=np.append(labels_test,testGenderMap[s])
print("Done")

Done


In [12]:
import sklearn 
from sklearn.neighbors import KNeighborsClassifier 

knn = KNeighborsClassifier(n_neighbors = 8)

knn.fit(x_train, labels_train)


Z=knn.predict(x_test)
print(sklearn.metrics.accuracy_score(labels_test, Z)) #Nicht ausgeglichenes Datenset => Akkuratheit kein gutes Evaluationsmaß

eval=sklearn.metrics.classification_report(labels_test, Z)
print(eval)

0.65
              precision    recall  f1-score   support

           F       0.69      0.75      0.72        12
           M       0.57      0.50      0.53         8

    accuracy                           0.65        20
   macro avg       0.63      0.62      0.63        20
weighted avg       0.64      0.65      0.65        20



C:\Users\ABleicher\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


### Teil 2

Anstatt für jede Session *Funktionale*, also Mittelwert-, Varianz-, Min-, Maxvektor auf den MFCCs zu berechnen und ggf. zu konkatenieren, können MFCCs mit dem KNN auch frame-weise klassifiziert werden. Dann erzeugt jeder MFCC-Vektor ein Klassifikationsergebnis. Anschließend müssen die frame-weisen Entscheidungen gemittelt werden um pro Session ein finales Ergebnis zu erhalten. 

Implementieren Sie dieses Vorgehen und vergleichen Sie die Ergebnisse mit denen aus Teil 1.

In [66]:
import statistics
import warnings

with warnings.catch_warnings(record=True):

    knn = KNeighborsClassifier(n_neighbors = 8)
    labels_train2=np.empty([0,1])
    x_train2=np.empty([0,20])

    speakers = os.listdir("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech\\train-clean")
    for s in speakers:
        sessions=os.listdir("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech\\train-clean\\"+s)
        for session in sessions:
            mfccMatrix=np.empty([0,20])
            files = os.listdir("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech\\train-clean\\"+s+"\\"+session)
            i=1
            for file in files:
                if(os.fsdecode(file).endswith('.flac')):
                    mfccMatrix=np.vstack([mfccMatrix, extract_feats("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech\\train-clean\\"+s+"\\"+session+"\\"+file)])
            x_train2=np.vstack([x_train2, mfccMatrix.mean(0)])
            labels_train2=np.append(labels_train2,trainGenderMap[s])
    print("Done")
    knn.fit(x_train2, labels_train2)

Done


In [74]:
import warnings

with warnings.catch_warnings(record=True):

    Z=np.empty([0,1])
    labels_test2=np.empty([0,1])
    speakers = os.listdir("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech\\test-clean-10spk")
    for s in speakers:
        sessions=os.listdir("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech\\test-clean-10spk\\"+s)
        for session in sessions:
            sessionLabels=np.empty([0,1])
            files = os.listdir("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech\\test-clean-10spk\\"+s+"\\"+session)
            i=1
            for file in files:
                if(os.fsdecode(file).endswith('.flac')):
                    sessionLabels=np.append(sessionLabels, knn.predict(extract_feats("C:\\Users\\ABleicher\\Desktop\\MachineLearning_SS23-main-6_speechfeatures-notebooks-data\\6_speechfeatures\\notebooks\\data\\LibriSpeech\\test-clean-10spk\\"+s+"\\"+session+"\\"+file)))
            Z=np.append(Z,statistics.mode(sessionLabels))
            labels_test2=np.append(labels_test2,testGenderMap[s])
    print("Done")
    print(Z)


Done
['F' 'F' 'F' 'F' 'F' 'M' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F'
 'F' 'F']


In [75]:
print(labels_test2)
print(Z)


print(sklearn.metrics.accuracy_score(labels_test2, Z)) #Nicht ausgeglichenes Datenset => Akkuratheit kein gutes Evaluationsmaß

eval=sklearn.metrics.classification_report(labels_test2, Z)
print(eval)

['F' 'F' 'F' 'M' 'M' 'M' 'M' 'F' 'F' 'M' 'M' 'M' 'F' 'M' 'F' 'F' 'F' 'F'
 'F' 'F']
['F' 'F' 'F' 'F' 'F' 'M' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F'
 'F' 'F']
0.65
              precision    recall  f1-score   support

           F       0.63      1.00      0.77        12
           M       1.00      0.12      0.22         8

    accuracy                           0.65        20
   macro avg       0.82      0.56      0.50        20
weighted avg       0.78      0.65      0.55        20

